## Dataset Specifications

In [1]:
# Variables
from torchsig.signals.signal_lists import TorchSigSignalLists
from torchsig.transforms.transforms import ComplexTo2D
import os
import numpy as np
from torch import Tensor
print("done")

done


In [2]:
root = "./datasets/PSK_v_AM_v_FM/classifier_example"
os.makedirs(root, exist_ok=True)
os.makedirs(root + "/train", exist_ok=True)
os.makedirs(root + "/val", exist_ok=True)
os.makedirs(root + "/test", exist_ok=True)
fft_size = 1024
num_iq_samples = fft_size ** 2
num_iq_samples_dataset = num_iq_samples
num_samples_per_class = 1000 # the number of samples for each modulation in the class list to create
percent_validation = 0.2 # the number of validation data points to generate as a percentage of the num of train data points
class_list = TorchSigSignalLists.all_signals
family_list = TorchSigSignalLists.family_list
print("done")

done


In [3]:
print(family_list)
family_list = ['psk', 'am', 'fm', 'ofdm']
print(family_list)
print(class_list)
class_list = ['bpsk','fm' , 'am-dsb-sc', 'ofdm-128', 'lfm_radar', 'chirpss', 'ook', '16ask', '64qam']
print(class_list)

['am', 'ask', 'chirp', 'fm', 'fsk', 'ofdm', 'ook', 'psk', 'qam', 'tone']
['psk', 'am', 'fm', 'ofdm']
['ook', '4ask', '8ask', '16ask', '32ask', '64ask', '2fsk', '2gfsk', '2msk', '2gmsk', '4fsk', '4gfsk', '4msk', '4gmsk', '8fsk', '8gfsk', '8msk', '8gmsk', '16fsk', '16gfsk', '16msk', '16gmsk', 'bpsk', 'qpsk', '8psk', '16psk', '32psk', '64psk', '16qam', '32qam', '32qam_cross', '64qam', '128qam_cross', '256qam', '512qam_cross', '1024qam', 'ofdm-64', 'ofdm-72', 'ofdm-128', 'ofdm-180', 'ofdm-256', 'ofdm-300', 'ofdm-512', 'ofdm-600', 'ofdm-900', 'ofdm-1024', 'ofdm-1200', 'ofdm-2048', 'fm', 'am-dsb-sc', 'am-dsb', 'am-lsb', 'am-usb', 'lfm_data', 'lfm_radar', 'chirpss', 'tone']
['bpsk', 'fm', 'am-dsb-sc', 'ofdm-128', 'lfm_radar', 'chirpss', 'ook', '16ask', '64qam']


In [4]:
num_classes = len(class_list)
num_samples_train = len(class_list) * num_samples_per_class # 
num_samples_val = int(len(class_list) * num_samples_per_class * percent_validation)
impairment_level = 0
seed = 123456789
 # IQ-based mod-rec only operates on 1 signal
num_signals_max = 1
num_signals_min = 1

# ComplexTo2D turns a IQ array of complex values into a 2D array, with one channel for the real component, while the other is for the imaginary component
transforms = [ComplexTo2D()]
print("done")

done


## Create Dataset

In [5]:
from torchsig.datasets.dataset_metadata import DatasetMetadata
from torchsig.datasets.datasets import TorchSigIterableDataset, StaticTorchSigDataset
from torchsig.utils.data_loading import WorkerSeedingDataLoader
from torchsig.utils.writer import DatasetCreator

dataset_metadata = DatasetMetadata(
    num_iq_samples_dataset = num_iq_samples_dataset,
    fft_size = fft_size,
    class_list = class_list,
    num_signals_max = num_signals_max,
    num_signals_min = num_signals_min,
)

train_dataset = TorchSigIterableDataset(dataset_metadata, transforms=transforms, target_labels=None)
val_dataset = TorchSigIterableDataset(dataset_metadata, transforms=transforms, target_labels=None)

train_dataloader = WorkerSeedingDataLoader(train_dataset, batch_size=1, collate_fn = lambda x: x)
val_dataloader = WorkerSeedingDataLoader(val_dataset, collate_fn = lambda x: x)

#print(f"Data shape: {data.shape}")
#print(f"Targets: {targets}")
# next(train_dataset)

dc = DatasetCreator(
    dataloader=train_dataloader,
    root = f"{root}/train",
    overwrite=True,
    dataset_length=num_samples_train
)
dc.create()


dc = DatasetCreator(
    dataloader=val_dataloader,
    root = f"{root}/val",
    overwrite=True,
    dataset_length=num_samples_val
)
dc.create()
print("done")

0it [00:00, ?it/s]

Deleted folder: datasets/PSK_v_AM_v_FM/classifier_example/train


0it [00:00, ?it/s]

Deleted folder: datasets/PSK_v_AM_v_FM/classifier_example/val
done


In [6]:
train_dataset = StaticTorchSigDataset(
    root = f"{root}/train",
    target_labels=["class_index"]
)
val_dataset = StaticTorchSigDataset(
    root = f"{root}/val",
    target_labels=["class_index"]
)

train_dataloader = WorkerSeedingDataLoader(train_dataset, batch_size=1, num_workers=127)
val_dataloader = WorkerSeedingDataLoader(val_dataset, num_workers=127)

print(train_dataset[0])
#print("done")
print(len(train_dataloader))
print(len(val_dataloader))
next(iter(train_dataloader))
print("done")

(array([[-0.05417608,  0.07383577,  0.09509687, ..., -0.03767033,
        -0.12575406, -0.00108993],
       [-0.02886894,  0.0069131 , -0.08293553, ...,  0.0399799 ,
        -0.10247673,  0.0880217 ]], dtype=float32), 0)
90000
18000
done


In [7]:
'''
train_data = []
train_label = []
for x in train_dataset:
    #hold = (x[0],x[1])
    train_data.append(x[0])
    train_label.append(x[1])
    
val_data = []
val_label = []
for x in val_dataset:
    #hold = (x[0],x[1])
    val_data.append(x[0])
    val_label.append(x[1])

print("done")
'''

'\ntrain_data = []\ntrain_label = []\nfor x in train_dataset:\n    #hold = (x[0],x[1])\n    train_data.append(x[0])\n    train_label.append(x[1])\n    \nval_data = []\nval_label = []\nfor x in val_dataset:\n    #hold = (x[0],x[1])\n    val_data.append(x[0])\n    val_label.append(x[1])\n\nprint("done")\n'

In [8]:
##Trying nesting the arrays
train_data = []
train_label = []
for x in train_dataset:
    #hold = (x[0],x[1])
    train_data.append([x[0]])
    train_label.append(x[1])
    
val_data = []
val_label = []
for x in val_dataset:
    #hold = (x[0],x[1])
    val_data.append([x[0]])
    val_label.append(x[1])

print("done")

done


In [9]:
print(train_data[0])
print(train_data[0][0])

[array([[-0.05417608,  0.07383577,  0.09509687, ..., -0.03767033,
        -0.12575406, -0.00108993],
       [-0.02886894,  0.0069131 , -0.08293553, ...,  0.0399799 ,
        -0.10247673,  0.0880217 ]], dtype=float32)]
[[-0.05417608  0.07383577  0.09509687 ... -0.03767033 -0.12575406
  -0.00108993]
 [-0.02886894  0.0069131  -0.08293553 ...  0.0399799  -0.10247673
   0.0880217 ]]


In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import os
import time

class Timer:
    """A simple context manager for timing code blocks."""
    def __init__(self, timings_dict, key):
        self.timings = timings_dict
        self.key = key
        self.start_time = None

    def __enter__(self):
        self.start_time = time.time()
        print(f"[{self.key}] starting...")
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        elapsed = time.time() - self.start_time
        self.timings[self.key] = elapsed
        print(f"[{self.key}] finished in {elapsed:.2f} seconds.")

def report_timings(timings):
    """Prints a summary of the timings."""
    print("\n--- Timing Report ---")
    total_time = 0
    for key, value in timings.items():
        print(f"  - {key}: {value:.2f} seconds")
        total_time += value
    print("---------------------")
    print(f"  Total Elapsed Time: {total_time:.2f} seconds")

def save_report(report_path, config, model_summary, history, final_accuracy, timings):
    """Saves a detailed report of the training session to a text file."""
    print(f"\n--- Saving Training Report to {report_path} ---")
    with open(report_path, 'w') as f:
        f.write("="*60 + "\n")
        f.write("           TRAINING SESSION REPORT\n")
        f.write("="*60 + "\n\n")
        
        # --- 1. Configuration ---
        f.write("--- 1. Training Configuration ---\n")
        for key, value in config.items():
            f.write(f"  - {key}: {value}\n")
        f.write("\n")

        # --- 2. Model Architecture ---
        f.write("--- 2. Model Architecture ---\n")
        f.write(model_summary + "\n\n")

        # --- 3. Training History ---
        f.write("--- 3. Training History (Epoch-wise) ---\n")
        epochs = range(1, len(history['train_loss']) + 1)
        header = f"{'Epoch':<7} | {'Train Loss':<12} | {'Train Acc (%)':<15} | {'Val Acc (%)':<13}\n"
        f.write(header)
        f.write("-" * len(header) + "\n")
        for i, epoch in enumerate(epochs):
            loss = history['train_loss'][i]
            train_acc = history['train_acc'][i]
            val_acc = history['val_acc'][i]
            f.write(f"{epoch:<7} | {loss:<12.4f} | {train_acc:<15.2f} | {val_acc:<13.2f}\n")
        f.write("\n")

        # --- 4. Final Performance ---
        f.write("--- 4. Final Performance on Test Set ---\n")
        f.write(f"  - Accuracy: {final_accuracy:.2f} %\n\n")

        # --- 5. Timing Report ---
        f.write("--- 5. Timing Report ---\n")
        total_time = 0
        for key, value in timings.items():
            f.write(f"  - {key}: {value:.2f} seconds\n")
            total_time += value
        f.write("---------------------\n")
        f.write(f"  Total Elapsed Time: {total_time:.2f} seconds\n")
    
    print("Report saved successfully.")

class DynamicCNN(nn.Module):
    """
    A dynamically configurable Convolutional Neural Network (CNN).
    The user can specify the architecture including convolutional layers,
    pooling layers, and fully connected layers.
    """
    def __init__(self, input_channels, num_classes, conv_layers_config, fc_layers_config, num_iq_samples):
        """
        Initializes the DynamicCNN model.

        Args:
            input_channels (int): Number of input channels (e.g., 3 for RGB images).
            num_classes (int): Number of output classes.
            conv_layers_config (list of tuples): Each tuple defines a convolutional block.
                Format: (out_channels, kernel_size, stride, padding, use_pooling)
                'use_pooling' is a boolean to add a MaxPool2d layer after the conv layer.
            fc_layers_config (list of int): A list where each integer is the number of
                neurons in a fully connected layer.
        """
        super(DynamicCNN, self).__init__()
        self.conv_layers = nn.ModuleList()
        self.pool_layers = nn.ModuleList()
        self.fc_layers = nn.ModuleList()

        in_channels = input_channels

        print("model intial in_channels: ", in_channels)

        # Create convolutional layers dynamically
        for out_channels, kernel_size, stride, padding, use_pooling in conv_layers_config:
            self.conv_layers.append(
                nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding)
            )
            if use_pooling:
                self.pool_layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
            else:
                self.pool_layers.append(nn.Identity()) # Placeholder if no pooling
            in_channels = out_channels

        # To determine the input size of the first fully connected layer,
        # we need to do a forward pass with a dummy tensor.
        # This is a common practice when the architecture is dynamic.
        with torch.no_grad():
            dummy_input = torch.zeros(1, input_channels, num_iq_samples[0], num_iq_samples[1]) # !!! Changeing this to account for shape of IQ data
            dummy_output = self._forward_conv(dummy_input)
            flattened_size = dummy_output.view(dummy_output.size(0), -1).shape[1]

        in_features = flattened_size

        print("in features for first FC layer: ", in_features)

        # Create fully connected layers dynamically
        for out_features in fc_layers_config:
            self.fc_layers.append(nn.Linear(in_features, out_features))
            in_features = out_features
        
        # Output layer
        self.output_layer = nn.Linear(in_features, num_classes)

    def _forward_conv(self, x):
        """Helper function to forward through conv layers only."""
        count = 0
        for conv, pool in zip(self.conv_layers, self.pool_layers):
            '''
            count+=1
            print("layer: ", count, " with input len", len(x), "and input: ", x)
            print("x[0]: ", x[0]) 
            print("len x[0]: ", len(x[0]))
            print("x[0][0]: ", x[0][0]) 
            print("len x[0][0]: ", len(x[0][0])) 
            '''
            x = torch.relu(conv(x))
            x = pool(x)
        return x

    def forward(self, x):
        """
        Defines the forward pass of the model.
        """
        '''
        print("start x: ", x) 
        print("len start x: ", len(x)) 
        print("start x[0]: ", x[0]) 
        print("len start x[0]: ", len(x[0]))
        print("start x[0][0]: ", x[0][0]) 
        print("len start x[0][0]: ", len(x[0][0])) 
        '''
        x = self._forward_conv(x)
        
        # Flatten the output for the fully connected layers
        x = x.view(x.size(0), -1)
        
        # Forward pass through fully connected layers
        for fc_layer in self.fc_layers:
            x = torch.relu(fc_layer(x))
            
        x = self.output_layer(x)
        return x

print("done")

done


In [11]:
def evaluate_model(model, loader, device='cpu'):
    """
    Evaluates the model's accuracy on a given dataset.
    Used for validation and training accuracy checks.
    """
    model.to(device)
    model.eval() # Set the model to evaluation mode
    
    correct = 0
    total = 0
    
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
    accuracy = 100 * correct / total
    return accuracy

def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10, device='cpu'):
    """
    Trains the PyTorch model and tracks history for plotting.
    """
    model.to(device)
    
    history = {
        'train_loss': [],
        'train_acc': [],
        'val_acc': []
    }
    
    total_training_time = 0
    print("Starting training...")
    for epoch in range(num_epochs):
        epoch_start_time = time.time()
        model.train() # Set the model to training mode
        running_loss = 0.0
        for i, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Zero the parameter gradients
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            # Backward pass and optimize
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()

        epoch_loss = running_loss / len(train_loader)
        
        # Evaluate on training and validation set after each epoch
        train_acc = evaluate_model(model, train_loader, device)
        val_acc = evaluate_model(model, val_loader, device)
        
        history['train_loss'].append(epoch_loss)
        history['train_acc'].append(train_acc)
        history['val_acc'].append(val_acc)
        
        epoch_duration = time.time() - epoch_start_time
        total_training_time += epoch_duration
        
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Train Acc: {train_acc:.2f}%, Val Acc: {val_acc:.2f}% -- Took {epoch_duration:.2f}s')
            
    print(f"Finished training. Total training time: {total_training_time:.2f}s")
    return history, total_training_time

def test_model(model, test_loader, device='cpu'):
    """
    Tests the PyTorch model.
    """
    model.to(device)
    model.eval() # Set the model to evaluation mode
    
    correct = 0
    total = 0
    
    print("Starting testing...")
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f'Accuracy of the model on the test images: {accuracy:.2f} %')
    return accuracy

def plot_results(history):
    """
    Plots training and validation metrics using matplotlib.
    """
    print("\n--- Plotting Results ---")
    
    try:
        import matplotlib.pyplot as plt
    except ImportError:
        print("matplotlib not found. Please install it ('pip install matplotlib') to plot results.")
        return

    epochs = range(1, len(history['train_loss']) + 1)

    plt.figure(figsize=(14, 6))

    # Plot Training & Validation Accuracy
    plt.subplot(1, 2, 1)
    plt.plot(epochs, history['train_acc'], 'o-', label='Training Accuracy', color='b')
    plt.plot(epochs, history['val_acc'], 'o-', label='Validation Accuracy', color='r')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy (%)')
    plt.legend()
    plt.grid(True)

    # Plot Training Loss
    plt.subplot(1, 2, 2)
    plt.plot(epochs, history['train_loss'], 'o-', label='Training Loss', color='g')
    plt.title('Training Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    
    plt.tight_layout()
    plt.savefig('training_results.png')
    print("Results plot saved as 'training_results.png'")
    # You can uncomment the line below to display the plot directly
    # plt.show()

def save_model(model, path="cnn_model.pth"):
    """
    Saves the trained model's state dictionary.
    Handles models wrapped in nn.DataParallel.
    """
    print(f"Saving model to {path}...")
    # If the model is wrapped in DataParallel, save the underlying model's state_dict
    if isinstance(model, nn.DataParallel):
        torch.save(model.module.state_dict(), path)
    else:
        torch.save(model.state_dict(), path)
    print("Model saved.")

def load_model(model_class, path, *args, **kwargs):
    """
    Loads a model's state dictionary from a file.
    Args:
        model_class: The class of the model to be instantiated.
        path (str): The path to the saved model file.
        *args, **kwargs: Arguments needed to instantiate the model class.
    """
    print(f"Loading model from {path}...")
    model = model_class(*args, **kwargs)
    model.load_state_dict(torch.load(path))
    print("Model loaded.")
    return model

def analyze_dataset(loader, dataset_name="Training"):
    """
    Performs and prints a statistical analysis on the dataset from a DataLoader.
    Calculates class distribution, image dimensions, and pixel value statistics.

    Args:
        loader (DataLoader): The DataLoader for the dataset to analyze.
        dataset_name (str): Name of the dataset (e.g., "Training", "Testing").
    """
    print(f"\n--- Analyzing {dataset_name} Dataset ---")

    if not hasattr(loader, 'dataset'):
        print("DataLoader does not have a 'dataset' attribute.")
        return

    dataset = loader.dataset
    
    # Handle datasets wrapped in torch.utils.data.Subset (from random_split)
    if isinstance(dataset, torch.utils.data.Subset):
        # Get labels and data from the original dataset using subset indices
        labels = [dataset.dataset.tensors[1][i] for i in dataset.indices]
        data_tensor = torch.stack([dataset.dataset.tensors[0][i] for i in dataset.indices])
        labels_tensor = torch.tensor(labels)
    elif isinstance(dataset, TensorDataset):
        # Handle regular TensorDataset
        data_tensor = dataset.tensors[0]
        labels_tensor = dataset.tensors[1]
    else:
        print(f"Dataset type {type(dataset).__name__} is not supported for analysis.")
        return

    if len(labels_tensor) == 0:
        print("No labels found to analyze.")
        return

    # 1. Class Distribution
    print("Class Distribution:")
    unique_classes, counts = torch.unique(labels_tensor, return_counts=True)
    class_dist = dict(zip(unique_classes.tolist(), counts.tolist()))
    
    for cls in sorted(class_dist.keys()):
        percentage = (class_dist[cls] / len(labels_tensor)) * 100
        print(f"  Class {cls}: {class_dist[cls]} samples ({percentage:.2f}%)")

    # 2. General Metrics
    num_samples = data_tensor.shape[0]
    print(f"\nTotal Samples: {num_samples}")

    if data_tensor.dim() == 4:  # Expected format: (N, C, H, W)
        num_channels = data_tensor.shape[1]
        height = data_tensor.shape[2]
        width = data_tensor.shape[3]
        print(f"Image Dimensions: {height}x{width}")
        print(f"Number of Channels: {num_channels}")

        # 3. Mean and Standard Deviation of pixel values per channel
        # This is useful for data normalization
        print("\nPixel Value Statistics (per channel):")
        # Reshape to (N*H*W, C) to calculate stats per channel easily
        pixels = data_tensor.permute(0, 2, 3, 1).reshape(-1, num_channels)
        mean = pixels.mean(axis=0)
        std = pixels.std(axis=0)

        for c in range(num_channels):
            print(f"  Channel {c}: Mean={mean[c]:.4f}, Std Dev={std[c]:.4f}")
    
    print("--------------------------------------")


def create_dummy_dataset(num_samples=1000, img_size=(64, 64), num_classes=10, channels=3):
    """
    Creates a dummy dataset for demonstration purposes.
    Returns:
        (DataLoader, DataLoader): train_loader, test_loader
    """
    print("Creating a dummy dataset...")
    # Generate random images and labels
    data = np.random.rand(num_samples, channels, img_size[0], img_size[1]).astype(np.float32)
    labels = np.random.randint(0, num_classes, num_samples).astype(np.int64)

    # Convert to PyTorch tensors
    tensor_x = torch.Tensor(data)
    tensor_y = torch.Tensor(labels).long()

    dataset = TensorDataset(tensor_x, tensor_y)
    
    # Split into training and testing
    train_size = int(0.8 * len(dataset))
    test_size = len(dataset) - train_size
    train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
    
    print("Dummy dataset created.")
    return train_loader, test_loader

print("done")

done


In [12]:
'''
hold_train, hold_test = create_dummy_dataset(num_samples=100, img_size=(1, 4096), num_classes=6, channels=2)

print(hold_train)
'''

'\nhold_train, hold_test = create_dummy_dataset(num_samples=100, img_size=(1, 4096), num_classes=6, channels=2)\n\nprint(hold_train)\n'

In [13]:
'''
print(len(hold_train.dataset[0]))
print((hold_train.dataset[0]))
print(len(hold_train.dataset[0][0]))
print((hold_train.dataset[0][0]))
'''

'\nprint(len(hold_train.dataset[0]))\nprint((hold_train.dataset[0]))\nprint(len(hold_train.dataset[0][0]))\nprint((hold_train.dataset[0][0]))\n'

## Format RF data into data_loaders

In [14]:
def Format_RF_dataset(data, labels, batch_size = 32, train_test_split = 0.2):
    """
    Formats labeled IQ data into dataloader
    Returns:
        (DataLoader, DataLoader): train_loader, test_loader
    """
    print("Formatting RF dataset...")
    # Generate random images and labels
    #data = np.random.rand(num_samples, channels, img_size[0], img_size[1]).astype(np.float32)
    #labels = np.random.randint(0, num_classes, num_samples).astype(np.int64)

    data = np.array(data)
    labels = np.array(labels)

    # Convert to PyTorch tensors
    tensor_x = torch.Tensor(data)
    tensor_y = torch.Tensor(labels).long()

    dataset = TensorDataset(tensor_x, tensor_y)
    
    # Split into training and testing
    train_size = int((1-train_test_split) * len(dataset))
    test_size = len(dataset) - train_size
    train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
    
    print("RF dataset loader created.")
    return train_loader, test_loader

print("done")

done


In [15]:
'''
train_loader, test_loader= Format_RF_dataset(train_data, train_label, batch_size = 32, train_test_split = 0.2)
val_loader,_ = Format_RF_dataset(val_data, val_label, batch_size = 32, train_test_split = 0.2)
'''

'\ntrain_loader, test_loader= Format_RF_dataset(train_data, train_label, batch_size = 32, train_test_split = 0.2)\nval_loader,_ = Format_RF_dataset(val_data, val_label, batch_size = 32, train_test_split = 0.2)\n'

In [16]:
'''
print(len(train_loader.dataset[0]))
print((train_loader.dataset[0]))
print(len(train_loader.dataset[0][0]))
print((train_loader.dataset[0][0]))
'''

'\nprint(len(train_loader.dataset[0]))\nprint((train_loader.dataset[0]))\nprint(len(train_loader.dataset[0][0]))\nprint((train_loader.dataset[0][0]))\n'

## Create Model

In [ ]:
#74% accuracy with output_channels = 16, batch size 32, lr = 0.0001 for PSKvAMvFM, 800 samples each, epoch = 30
#77% w output_channels = 64 , batch_size = 8, epoch = 50
#74% @ epochs = 100, 323.327 MB
#67% w output_channels = 128, batch = 8, epoch = 30, 1293.217 MB
#73% w output_channels = 256, batch = 8, epoch = 30, 5172.685 MB
### All above with one FC layer = [output_channels*2*2*2*2, output_channels]
#NOTE: FC layer should not scale 1:1 with the size of CNN layers, it quickly becomes too large
#75.5% w output_channels = 256, batch_size = 32, epoch = 30, fc_config = [output_channels*2*2, output_channels], 3249.673 MB
#output channel above 256 seems bad and slow
#81.18 % test accuracy, 100% train accuracy, 500 epoch, same config as above

num_classes = len(class_list)
num_iq_samples
output_channels = 32
batch_size = 32
epochs = 100
lr = 0.0001 #learning rate
print(f"num classes {num_classes}")
print(f"input length {num_iq_samples}")
print(f"output channels {output_channels}")

if __name__ == '__main__':
    # --- Configuration ---
    config = {
        'INPUT_CHANNELS': 1, #### going to try treating IQ as one input channel of 2d data input size *2
        'NUM_CLASSES': num_classes,
        'IMAGE_SIZE': (2,num_iq_samples),
        'DEVICE': str(torch.device("cuda" if torch.cuda.is_available() else "cpu")),
        'MODEL_SAVE_PATH': "custom_cnn.pth",
        'REPORT_SAVE_PATH': "training_report.txt",
        'NUM_EPOCHS': epochs,
        'LEARNING_RATE': lr #good learning rates: 0.0001
    }

    # Define a custom CNN architecture
    # Format: (out_channels, kernel_size, stride, padding, use_pooling)

    '''
    conv_config = [
        (output_channels, 2, 1, 0, True),   # 1st Conv Layer -> MaxPool ##!!!removed padding ###!!!set kernel size to 1
        (output_channels*2, 2, 1, 0, True),   # 2nd Conv Layer -> MaxPool
        (output_channels*2*2, 2, 1, 0, False)   # 3rd Conv Layer
    ]
    '''

    '''
    ##!!!!This one works!!!!
    conv_config = [
        (16, 2, 1, 1, True),   # 1st Conv Layer -> MaxPool
        (32, 2, 1, 1, True),   # 2nd Conv Layer -> MaxPool
        (64, 2, 1, 1, False)   # 3rd Conv Layer
    ]
    '''

    conv_config = [
        (output_channels, 2, 2, 1, True),   # 1st Conv Layer -> MaxPool
        (output_channels*2, 1, 1, 1, True),   # 2nd Conv Layer -> MaxPool
        (output_channels*2, 2, 1, 1, True),   # 2nd Conv Layer -> MaxPool
        (output_channels*2*2, 2, 1, 1, True),   # 2nd Conv Layer -> MaxPool
        (output_channels*2*2, 1, 1, 1, True),   # 2nd Conv Layer -> MaxPool
        (output_channels*2*2*2, 2, 1, 1, True),   # 2nd Conv Layer -> MaxPool
        (output_channels*2*2*2*2, 2, 1, 1, True),   # 2nd Conv Layer -> MaxPool
        (output_channels*2*2*2*2*2, 1, 1, 1, True),   # 2nd Conv Layer -> MaxPool
        (output_channels*2*2*2*2*2, 1, 1, 1, True),   # 2nd Conv Layer -> MaxPool
        (output_channels*2*2*2*2*2, 2, 1, 1, True),   # 2nd Conv Layer -> MaxPool
        (output_channels*2*2*2*2*2, 2, 1, 1, False),   # 2nd Conv Layer -> MaxPool
        (output_channels*2*2*2*2*2, 1, 1, 1, True),   # 2nd Conv Layer -> MaxPool
        (output_channels*2*2*2*2*2, 1, 1, 1, True),   # 2nd Conv Layer -> MaxPool
        (output_channels*2*2*2*2*2, 2, 1, 1, True),   # 2nd Conv Layer -> MaxPool
        (output_channels*2*2*2*2*2, 2, 1, 1, False),   # 2nd Conv Layer -> MaxPool
        
    ]

    

    # Format: list of output features for each fully connected layer
    #fc_config = [output_channels*2*2*2*2, output_channels] ##!!!NOTE: this is good for smaller models
    fc_config = [output_channels*2*2*2, output_channels*2, output_channels]
    
    # --- 1. Create a dummy dataset ---
    # In a real scenario, you would replace this with your own data loading logic.
    # Ensure your data is in the format of (N, C, H, W)
    # N: Number of samples, C: Channels, H: Height, W: Width
    
    timings = {} # Dictionary to store timings of each step

    with Timer(timings, "1. Dataset Creation & Analysis"):
        train_loader, test_loader= Format_RF_dataset(train_data, train_label, batch_size = batch_size, train_test_split = 0.2)
        val_loader,_ = Format_RF_dataset(val_data, val_label, batch_size = batch_size, train_test_split = 0.0)#train test split set to 0 for val set
        # --- 1.5. Analyze the training data ---
        analyze_dataset(train_loader, dataset_name="Training")
    
    # --- 1.6. Create a validation split ---
    with Timer(timings, "2. Validation Split"):
        # In a real-world scenario, you would have a separate validation dataset.
        # Here, we split the original training data for demonstration.
        print("\n--- Creating Validation Split ---")
        #train_dataset = train_loader.dataset
        #val_split = 0.2
        #dataset_size = len(train_dataset)
        #val_size = int(val_split * dataset_size)
        #train_size = dataset_size - val_size
        
        #new_train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_size, val_size])
        
        # Create new DataLoaders for the split data
        #new_train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
        #val_loader = DataLoader(val_data, batch_size=32, shuffle=False)
        print(f"Split training data into {len(train_data)} training samples and {len(val_data)} validation samples.")

    

    
    # --- 2. Create the CNN model ---
    with Timer(timings, "3. Model Creation"):
        print("\n--- Creating Model ---")
        my_cnn = DynamicCNN(
            input_channels=config['INPUT_CHANNELS'], 
            num_classes=config['NUM_CLASSES'],
            conv_layers_config=conv_config,
            fc_layers_config=fc_config,
            num_iq_samples = config['IMAGE_SIZE']
        )
        
        # Check for multiple GPUs and wrap the model with DataParallel for multi-GPU training
        if torch.cuda.is_available() and torch.cuda.device_count() > 1:
            print(f"Using {torch.cuda.device_count()} GPUs for training!")
            my_cnn = nn.DataParallel(my_cnn)

        model_summary_str = str(my_cnn)
        print("Model Architecture:")
        print(model_summary_str)

    # --- 3. Define Loss Function and Optimizer ---
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(my_cnn.parameters(), lr=config['LEARNING_RATE'])

    # --- 4. Train the model ---
    print("\n--- Training Model ---")
    history, training_time = train_model(
        my_cnn, 
        train_loader, 
        val_loader, 
        criterion, 
        optimizer, 
        num_epochs=config['NUM_EPOCHS'], 
        device=config['DEVICE']
    )
    timings["4. Model Training"] = training_time


    # --- 5. Plot training results ---
    with Timer(timings, "5. Plotting Results"):
        plot_results(history)

    # --- 6. Test the model on the unseen test set ---
    with Timer(timings, "6. Final Model Testing"):
        print("\n--- Testing Model ---")
        final_test_accuracy = test_model(my_cnn, test_loader, device=config['DEVICE'])

    # --- 7. Save the model ---
    with Timer(timings, "7. Saving Model"):
        print("\n--- Saving Model ---")
        save_model(my_cnn, path=config['MODEL_SAVE_PATH'])

    # --- 8. Load the model ---
    with Timer(timings, "8. Loading Model"):
        print("\n--- Loading Model ---")
        # We need to provide the same configuration to instantiate the model class
        # before loading the saved weights.
        loaded_model = load_model(
            DynamicCNN,
            path=config['MODEL_SAVE_PATH'],
            input_channels=config['INPUT_CHANNELS'],
            num_classes=config['NUM_CLASSES'],
            conv_layers_config=conv_config,
            fc_layers_config=fc_config,
            num_iq_samples = config['IMAGE_SIZE']
        )
        print("Loaded Model Architecture:")
        print(loaded_model)

    # --- 9. Test the loaded model to verify it's working ---
    with Timer(timings, "9. Testing Loaded Model"):
        print("\n--- Testing Loaded Model ---")
        test_model(loaded_model, test_loader, device=config['DEVICE'])
    
    # --- 10. Report Timings ---
    report_timings(timings)
    # Calculate parameter size
    param_size = sum(param.nelement() * param.element_size() for param in loaded_model.parameters())
    
    
    # Calculate buffer size
    buffer_size = sum(buffer.nelement() * buffer.element_size() for buffer in loaded_model.buffers())
    
    # Total size in MB
    total_size_mb = (param_size + buffer_size) / 1024**2
    print(f"Model size: {total_size_mb:.3f} MB")

    model_summary_str = model_summary_str +"\n"+(f"Model size: {total_size_mb:.3f} MB")

    # --- 11. Save Final Report ---
    with Timer(timings, "10. Saving Report"):
        save_report(
            report_path=config['REPORT_SAVE_PATH'],
            config=config,
            model_summary=model_summary_str,
            history=history,
            final_accuracy=final_test_accuracy,
            timings=timings
        )

num classes 9
input length 1048576
output channels 32
[1. Dataset Creation & Analysis] starting...
Formatting RF dataset...
RF dataset loader created.
Formatting RF dataset...
RF dataset loader created.

--- Analyzing Training Dataset ---
Class Distribution:
  Class 0: 8056 samples (11.19%)
  Class 1: 8105 samples (11.26%)
  Class 2: 7920 samples (11.00%)
  Class 3: 7958 samples (11.05%)
  Class 4: 8121 samples (11.28%)
  Class 5: 7906 samples (10.98%)
  Class 6: 7855 samples (10.91%)
  Class 7: 7966 samples (11.06%)
  Class 8: 8113 samples (11.27%)

Total Samples: 72000
Image Dimensions: 2x1048576
Number of Channels: 1

Pixel Value Statistics (per channel):
  Channel 0: Mean=0.0002, Std Dev=353.4129
--------------------------------------
[1. Dataset Creation & Analysis] finished in 556.87 seconds.
[2. Validation Split] starting...

--- Creating Validation Split ---
Split training data into 90000 training samples and 18000 validation samples.
[2. Validation Split] finished in 0.00 seco

In [ ]:
#todo
'''
1) make everything a script
2) name the files dynamically with the experimental hyper-params
3) be able to loop through training rounds
4) save models with names relating to performance (only need to save models for extrememly long training times, for normal experiments can
just keep the hyper-parameters and data generation specifics and re-create the experiment if needed.
5) save all the training results and model names in a centralized location
6) spectrograms!!!
7) Embedding model
8) Add residual layers from gemini code
9) Add GUI???
'''